
The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program.


In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

Loading the fashion mnist data.

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Defining the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.


In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`.

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 23s]
val_accuracy: 0.8723333477973938

Best val_accuracy So Far: 0.8912500143051147
Total elapsed time: 00h 13m 58s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.001.



## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 6s 3ms/step - loss: 0.4949 - accuracy: 0.8240 - val_loss: 0.4050 - val_accuracy: 0.8540
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3684 - accuracy: 0.8669 - val_loss: 0.3733 - val_accuracy: 0.8630
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3309 - accuracy: 0.8786 - val_loss: 0.3670 - val_accuracy: 0.8655
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3068 - accuracy: 0.8865 - val_loss: 0.3401 - val_accuracy: 0.8783
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2848 - accuracy: 0.8951 - val_loss: 0.3192 - val_accuracy: 0.8850
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2720 - accuracy: 0.8987 - val_loss: 0.3461 - val_accuracy: 0.8767
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2559 - accuracy: 0.9047 - val_loss: 0.3137 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/18
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4942 - accuracy: 0.8240 - val_loss: 0.4032 - val_accuracy: 0.8513
Epoch 2/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3729 - accuracy: 0.8635 - val_loss: 0.3710 - val_accuracy: 0.8689
Epoch 3/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3317 - accuracy: 0.8786 - val_loss: 0.3325 - val_accuracy: 0.8794
Epoch 4/18
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3070 - accuracy: 0.8871 - val_loss: 0.3408 - val_accuracy: 0.8783
Epoch 5/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2856 - accuracy: 0.8939 - val_loss: 0.3473 - val_accuracy: 0.8738
Epoch 6/18
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2699 - accuracy: 0.8986 - val_loss: 0.3246 - val_accuracy: 0.8803
Epoch 7/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2578 - accuracy: 0.9037 - val_loss: 0.3062 - val_accuracy:

Evaluating the hypermodel on the test data.

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.3798 - accuracy: 0.8855
[test loss, test accuracy]: [0.37982237339019775, 0.8855000138282776]
